In [1]:
import os
from pathlib import Path

In [2]:
TIMEFRAME = "15min"
PAIR = "BTCUSDT"
DATA_PATH = Path.home() / "data" / PAIR
TMP_PATH = Path("./tmp/").resolve()
CONFIG_PATH = Path("./config/").resolve()
LOG_PATH = Path("./log/").resolve()
DATA_PATH.mkdir(exist_ok=True, parents=True)
TMP_PATH.mkdir(exist_ok=True, parents=True)
CONFIG_PATH.mkdir(exist_ok=True, parents=True)
LOG_PATH.mkdir(exist_ok=True, parents=True)

Save: OHLCV + Open Interest from Bybit API

In [3]:
import ccxt
from dotenv import load_dotenv
load_dotenv(verbose=True)
dotenv_path = Path.home() / ".env"
load_dotenv(dotenv_path)
exchange = ccxt.bybit()
exchange.apiKey = os.environ["BYBIT_API_KEY"]
exchange.secret = os.environ["BYBIT_SECRET"]
exchange.options["timeDifference"] = 5000

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
ohlcv = exchange.fetch_ohlcv(PAIR, "15m", limit=200)
oi = exchange.fetch_open_interest_history(PAIR, "15min", limit=200)
oi = [[int(d["info"]["timestamp"]) * 1000, float(d["info"]["open_interest"])] for d in oi]
assert (len(ohlcv) == len(oi))
assert (ohlcv[0][0] == oi[0][0] and ohlcv[-1][0] == oi[-1][0])
data = [a + b[1:] for a, b in zip(ohlcv, oi)]
df = pd.DataFrame(data, columns=["Datetime", "Open", "High", "Low", "Close", "Volume", "OpenInterest"])
df["Datetime"] = pd.to_datetime(df["Datetime"], unit="ms")
df = df.set_index("Datetime")
df.head()

,Open,High,Low,Close,Volume,OpenInterest
Datetime,,,,,,
2022-08-27 03:45:00,20227.5,20239.5,20191.5,20193.5,611.301,48830.093
2022-08-27 04:00:00,20193.5,20213.5,20153.5,20158.5,1216.345,48765.072
2022-08-27 04:15:00,20158.5,20191.5,20140.0,20173.5,1128.009,48988.243
2022-08-27 04:30:00,20173.5,20190.0,20148.5,20156.0,414.180,49049.690
2022-08-27 04:45:00,20156.0,20168.0,20119.0,20140.0,1054.285,49118.451


In [6]:
from scripts.extract_features import attach_features
df = attach_features(df)
oi = df["OpenInterest"].apply(np.log1p)
for ts in [1, 5, 10, 20]:
    df[f"feature_oi_log_return_{ts}"] = oi.diff(ts)

df.head()

,Open,High,Low,Close,Volume,OpenInterest,feature_candle_value,feature_candle_value_mean_10,feature_candle_value_mean_20,feature_candle_value_mean_5,...,feature_upper_shadow_mean_20,feature_upper_shadow_mean_5,feature_volatility_10,feature_volatility_20,feature_volatility_3,feature_volatility_5,feature_oi_log_return_1,feature_oi_log_return_5,feature_oi_log_return_10,feature_oi_log_return_20
Datetime,,,,,,,,,,,,,,,,,,,,,
2022-08-27 08:45:00,20237.0,20237.0,20168.0,20189.5,1722.287,49248.153,-0.688406,0.094071,-0.009505,-0.178576,...,0.001926,0.000963,0.001674,0.003029,0.001748,0.001889,NaN,NaN,NaN,NaN
2022-08-27 09:00:00,20189.5,20199.0,20162.0,20170.5,851.884,49270.132,-0.513514,0.026675,-0.006014,-0.427016,...,0.001860,0.001216,0.001709,0.003010,0.000753,0.001239,0.000446,NaN,NaN,NaN
2022-08-27 09:15:00,20170.5,20170.5,20132.0,20142.0,968.156,49338.098,-0.740260,-0.123740,-0.057591,-0.465734,...,0.001886,0.001281,0.001729,0.003022,0.000717,0.001263,0.001378,NaN,NaN,NaN
2022-08-27 09:30:00,20142.0,20175.0,20137.0,20166.0,953.055,49406.978,0.631579,-0.022121,-0.004927,-0.446735,...,0.001824,0.001366,0.001743,0.003029,0.001388,0.001305,0.001395,NaN,NaN,NaN
2022-08-27 09:45:00,20166.0,20166.0,20138.0,20145.0,612.418,49564.418,-0.750000,-0.174743,-0.026101,-0.412120,...,0.001807,0.001332,0.001513,0.003033,0.001409,0.001300,0.003181,NaN,NaN,NaN


In [7]:
import json
with open(CONFIG_PATH / "trainer.json", "r") as f:
    config = json.load(f)

In [16]:
p = DATA_PATH / "ohlcv_with_features"
config["env_config"]["observer"]["kwargs"]["df_paths"] = {
    "15min": str(p / "15T.pkl"),
    "1h": str(p / "1H.pkl"),
    "4h": str(p / "4H.pkl"),
}
config["env_config"]["observer"]["kwargs"]["step_multi"] = {
    "15min": 1,
    "1h": 4,
    "4h": 16,
}
config["evaluation_config"] = {}
config

{'env': rl_bot.envs.environment.TradingEnv,
 'env_config': {'fee': 0.0001,
  'observer': {'type': 'MultiTimeframeObserver',
   'kwargs': {'df_paths': {'15min': '/home/napnel/data/BTCUSDT/ohlcv_with_features/15T.pkl',
     '1h': '/home/napnel/data/BTCUSDT/ohlcv_with_features/1H.pkl',
     '4h': '/home/napnel/data/BTCUSDT/ohlcv_with_features/4H.pkl'},
    'step_multi': {'15min': 1, '1h': 4, '4h': 16},
    'window_size': 30}},
  'actions': {'type': 'MarketOrder', 'kwargs': {}},
  'rewards': {'type': 'DSR', 'kwargs': {'window_size': 30}},
  'informer': {'type': 'PrivateInformer', 'kwargs': {}},
  'stopper': {'type': 'DrawdownStopper',
   'kwargs': {'allowable_drawdown': 0.5}}},
 'evaluation_config': {},
 'evaluation_interval': None,
 'evaluation_duration': 1,
 'evaluation_parallel_to_training': False,
 'min_train_timesteps_per_reporting': 10000,
 'min_sample_timesteps_per_reporting': None,
 'min_time_s_per_reporting': None,
 'log_level': 'WARN',
 'framework': 'torch',
 'callbacks': None,
 

In [17]:
from rl_bot.envs.environment import TradingEnv
env = TradingEnv(config["env_config"])

In [18]:
# done = False
# obs = env.reset()
# rewards, steps = 0, 0
# while not done:
#     action = env.action_space.sample()
#     obs, reward, done, info = env.step(action)
#     rewards += reward
#     steps += 1

# print(steps, rewards)

In [19]:
from rl_bot.models.mtsfc_network import MTSFCNetwork
from ray.rllib.models import ModelCatalog
ModelCatalog.register_custom_model("MTSFCNetwork", MTSFCNetwork)

In [20]:
config["env"] = TradingEnv
config["model"] = {
    "custom_model": "MTSFCNetwork",
    "fcnet_hiddens": [64, 64],
    "fcnet_activation": "relu",
    "post_fcnet_hiddens": [256, 256],
    "post_fcnet_activation": "relu",
}
config["num_workers"] = 0
# config["_disable_preprocessor_api"] =  False

In [21]:
from ray.rllib.agents import dqn, pg
# agent = dqn.DQNTrainer(config)
agent = pg.PGTrainer(config)

In [25]:
obs = env.reset()
done = False
steps = 0
record = pd.DataFrame(columns=["tf", "Close"])
while not done:
    action = agent.compute_action(obs)
    obs, reward, done, info = env.step(action)
    print(env.observer.steps)
    for key, df in env.observer.ohlcv.items():
        print(df.iloc[env.observer.steps[key]])
    steps += 1
    if steps > 10:
        break

{'15min': 31, '1h': 30, '4h': 30}
Open      29534.000000
High      29534.000000
Low       29199.000000
Close     29276.000000
Volume      466.147003
Name: 2021-01-01 21:45:00+09:00, dtype: float32
Open      32612.500000
High      32935.000000
Low       32612.000000
Close     32777.000000
Volume     1067.177979
Name: 2021-01-03 11:00:00+09:00, dtype: float32
Open      38988.000000
High      40837.750000
Low       38727.000000
Close     40461.000000
Volume     3371.290039
Name: 2021-01-09 16:00:00+09:00, dtype: float32
{'15min': 32, '1h': 30, '4h': 30}
Open      29271.500000
High      29407.599609
Low       29264.500000
Close     29407.599609
Volume      120.640999
Name: 2021-01-01 22:00:00+09:00, dtype: float32
Open      32612.500000
High      32935.000000
Low       32612.000000
Close     32777.000000
Volume     1067.177979
Name: 2021-01-03 11:00:00+09:00, dtype: float32
Open      38988.000000
High      40837.750000
Low       38727.000000
Close     40461.000000
Volume     3371.290039
Na

In [23]:
agent.get_policy().model.flatten

{0: FullyConnectedNetwork(
   (_hidden_layers): Sequential(
     (0): SlimFC(
       (_model): Sequential(
         (0): Linear(in_features=1590, out_features=64, bias=True)
         (1): ReLU()
       )
     )
     (1): SlimFC(
       (_model): Sequential(
         (0): Linear(in_features=64, out_features=64, bias=True)
         (1): ReLU()
       )
     )
   )
   (_value_branch_separate): Sequential(
     (0): SlimFC(
       (_model): Sequential(
         (0): Linear(in_features=1590, out_features=64, bias=True)
         (1): ReLU()
       )
     )
     (1): SlimFC(
       (_model): Sequential(
         (0): Linear(in_features=64, out_features=64, bias=True)
         (1): ReLU()
       )
     )
   )
   (_value_branch): SlimFC(
     (_model): Sequential(
       (0): Linear(in_features=64, out_features=1, bias=True)
     )
   )
 ),
 1: FullyConnectedNetwork(
   (_hidden_layers): Sequential(
     (0): SlimFC(
       (_model): Sequential(
         (0): Linear(in_features=1590, out_featur